In [1]:
import pandas as pd
import sklearn
print(sklearn.__version__)


1.6.1


In [2]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
num_columns = df.shape[1]
print(num_columns)




19


In [5]:
import pandas as pd

# Make sure both columns are datetime objects
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Calculate trip duration
df['trip_duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

# Convert duration to minutes
df['trip_duration_min'] = df['trip_duration'].dt.total_seconds() / 60

std_dev = ((df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60).std()

print(f"Standard deviation of trip duration (minutes): {std_dev:.2f}")

# Define outliers: durations less than 1 minute or greater than 60 minutes
outlier_condition = (df['trip_duration_min'] < 1) | (df['trip_duration_min'] > 60)

# Number of non-outliers
num_non_outliers = (~outlier_condition).sum()

# Total rows
total_rows = len(df)

# Calculate percentage left after removing outliers
percentage_left = (num_non_outliers / total_rows) * 100
df = df[~outlier_condition].copy()


print(f"Percentage of data left after removing outliers: {percentage_left:.2f}%")


Standard deviation of trip duration (minutes): 42.59
Percentage of data left after removing outliers: 98.12%


In [6]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

# Step 1: Select and cast categorical columns to string
subset = df[['PULocationID', 'DOLocationID']].astype(str)


# Step 2: Convert to list of dictionaries
list_of_dicts = subset.to_dict(orient='records')


# Step 3: Initialize DictVectorizer and fit on training data
vec = DictVectorizer(sparse=True)
feature_matrix = vec.fit_transform(list_of_dicts)   # Training features


# Step 4: Output shapes
print("Train feature matrix shape:", feature_matrix.shape)
print("Sample train row (0):", feature_matrix[0])


Train feature matrix shape: (3009173, 515)
Sample train row (0):   (np.int32(0), np.int32(43))	1.0
  (np.int32(0), np.int32(325))	1.0


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Response variable (target)
y = df['trip_duration_min'].values

# Initialize linear regression model
model = LinearRegression()

# Train model on the feature matrix
model.fit(feature_matrix, y)

# Predict on training data
y_pred = model.predict(feature_matrix)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"RMSE on training data: {rmse:.4f} minutes")


RMSE on training data: 7.6493 minutes


In [8]:


# Step 2: Compute trip duration in minutes
df2['tpep_pickup_datetime'] = pd.to_datetime(df2['tpep_pickup_datetime'])
df2['tpep_dropoff_datetime'] = pd.to_datetime(df2['tpep_dropoff_datetime'])
df2['trip_duration'] = df2['tpep_dropoff_datetime'] - df2['tpep_pickup_datetime']
df2['trip_duration_min'] = df2['trip_duration'].dt.total_seconds() / 60

# Step 3: Remove outliers
df2 = df2[(df2['trip_duration_min'] >= 1) & (df2['trip_duration_min'] <= 60)]








In [ ]:
# Step 4: Prepare features (recast to string for one-hot encoding)
df2_features = df2[['PULocationID', 'DOLocationID']].astype(str)
X_val = vec.transform(df2_features.to_dict(orient='records'))  # Use existing DictVectorizer



In [ ]:
# Step 5: Predict and calculate RMSE
y_val = df2['trip_duration_min'].values
y_pred = model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

print(f"RMSE on validation data: {rmse_val:.4f} minutes")